In [1]:
import re
def clean_text(txt):
    cleanText = re.sub('http\S+\s', ' ', txt)
    #http-only select it,http\S-select http+next one cherecter, http\S+ -means full address,\s-next space.
    cleanText = re.sub('RT|cc', ' ', cleanText)
    #These are often used in tweets and are common in text data from social media.
    cleanText = re.sub('#\S+\s', ' ', cleanText)
    cleanText = re.sub('@\S+', '  ', cleanText)
    cleanText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleanText)
    #This line removes special characters such as punctuation, excluding whitespaces
    cleanText = re.sub(r'[^\x00-\x7f]', ' ', cleanText)
    #This line removes non-ASCII characters
    cleanText = re.sub('\s+', ' ', cleanText)
    #This line substitutes sequences of whitespace characters with a single whitespace character.
    #This ensures that multiple consecutive whitespace characters are reduced to just one.
    return cleanText

In [2]:
input_text = "Hi, how are you? Today is the first day of the week."
user_input = clean_text(input_text)
print(user_input)

Hi how are you Today is the first day of the week 


In [3]:
!pip install transformers

In [4]:
input_text = "Hi, how are you? Today is the first day of the week."
user_input = clean_text(input_text)
print(user_input)

Hi how are you Today is the first day of the week 


In [5]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

# Preprocess text (username and link placeholders)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)
text = "I love you very much"
text = clean_text(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

g:\DATA SCIENCE-25\Generative AI\generation_app\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
g:\DATA SCIENCE-25\Generative AI\generation_app\venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSeque

1) positive 0.9687
2) neutral 0.0249
3) negative 0.0064


In [6]:
from transformers import pipeline

# Load the text generation pipeline with GPT-2 model
text_generator = pipeline("text-generation", model="openai-community/gpt2")

# Define functions to generate prompts for different types of comments
def generate_friendly_prompt(input_sentence):
    return f"That's fantastic! {input_sentence}"

def generate_funny_prompt(input_sentence):
    return f"You won't believe it, but {input_sentence}"

def generate_congratulating_prompt(input_sentence):
    return f"Congratulations on {input_sentence}"

def generate_questioning_prompt(input_sentence):
    return f"I'm curious, {input_sentence}"

def generate_disagreement_prompt(input_sentence):
    return f"Sorry, I have to disagree. {input_sentence}"

# Get user input sentence
#user_sentence = input("Enter your sentence: ")

# Define prompts for different types of comments
prompts = {
    "Friendly": generate_friendly_prompt(user_input),
    "Funny": generate_funny_prompt(user_input),
    "Congratulating": generate_congratulating_prompt(user_input),
    "Questioning": generate_questioning_prompt(user_input),
    "Disagreement": generate_disagreement_prompt(user_input)
}

# Generate comments for each type
generated_comments = {}
for comment_type, prompt in prompts.items():
    generated_comment = text_generator(prompt, max_length=50, num_return_sequences=1)[0]['generated_text']
    generated_comments[comment_type] = generated_comment

# Print the generated comments for each type
print("\nGenerated Comments:")
for comment_type, comment in generated_comments.items():
    print(f"{comment_type}:")
    print(comment.strip())
    print()



g:\DATA SCIENCE-25\Generative AI\generation_app\venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` t


Generated Comments:
Friendly:
That's fantastic! Hi how are you Today is the first day of the week  of being a vegetarian for your own personal reasons! We've decided to start off by taking a look at three of your favourite ways to feed your animals.

Funny:
You won't believe it, but Hi how are you Today is the first day of the week  for us, a new start for you. Thank You very much for taking the time to speak with us today. It's a great day,

Congratulating:
Congratulations on Hi how are you Today is the first day of the week  from all of you.
Hi. A short recap with the first day of the week and the start of a month, now that you have all been invited we need

Questioning:
I'm curious, Hi how are you Today is the first day of the week "

"Hello and welcome, we live in a small county in south eastern England. I'm on your mobile so don't be rude please come in today

Disagreement:
Sorry, I have to disagree. Hi how are you Today is the first day of the week  of the concert! As is so freq